# House Prices

## Autor

Erick Melo Vidal de Olvieira - 190027355

## Objetivo

Este modelo tem como objetivo prever o preço de uma casa a partir da defnição de 79 variáveis diferentes, utilizando a competição [House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques).

## Importando o DataSet

Como o DataSet posssui muitas variáveis e não será possível visualizar 79 colunas neste nootebook, a descrição de cada uma delas e seus respectivos significados podem ser encontrados neste [link](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data).

In [6]:
# Importar as bibliotecas necessárias
import os
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

path = Path('../input/house-prices-advanced-regression-techniques')

df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


Primeiramente, devemos separar o nosso dataset entre variáveis dependentes e independentes. A variável dependente será o que queremos prever, que é o SalePrice nesse caso. Já as variáveis independentes serão todas as outras colunas, com exceção da coluna "Id", pois ela não interfere no preço final da casa.

In [90]:
t_dep = df["SalePrice"]
t_indep = df.drop(["Id", "SalePrice"], axis=1)
test = test_df.drop(["Id"], axis=1)

Em seguida, é possivel notar que existem muitos valores NaN no dataset. Porém, como cada valor do dataset será multiplicado por um coeficiente posteriormente, esses valores NaN devem ser tratados para que essa multiplicação seja possível.

Uma das maneiras de fazer este tratamento é substituir todos esses valores por 0, mas não é uma maneira tão eficiente. Neste modelo então, eu irei substituí-los pela moda (valor que mais se repete) da sua respectiva coluna.

Para encontrar essa moda será utilizada a função `mode()`. Porém, se a moda empatar, ou seja, se existirem dois ou mais valores que se repetem o mesmo número de vezes na coluna, essa função retornará um vetor com os números que empataram. Então o `iloc[0]` será usado para selecionar apenas um valor.

In [26]:
modes = df.mode().iloc[0]
modes

Id                      1
MSSubClass           20.0
MSZoning               RL
LotFrontage          60.0
LotArea            7200.0
                   ...   
MoSold                6.0
YrSold             2009.0
SaleType               WD
SaleCondition      Normal
SalePrice        140000.0
Name: 0, Length: 81, dtype: object

Para substituir os valores utilizando o método fillna() será necessário deixar o parâmetro inplace como true, para que as mudanças ocorram no próprio DataFrame, sem a necessidade de retornar um valor.

In [27]:
t_indep.fillna(modes, inplace=True)
test.fillna(modes, inplace=True)

Além disso, também existem algumas colunas que são categóricas, e por isso não será possível multiplicar seus valores por um coeficiente, portanto essas colunas também precisarão passar por um tratamento.

In [36]:
df.select_dtypes(include=["object"])

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


Para o tratamento dessas colunas será aplicada a função `lambda` em cada uma delas. O objetivo dessa operação é converter as colunas categóricas em colunas numéricas, atribuindo um código único a cada categoria.

In [28]:
categorical_columns = t_indep.select_dtypes(include=["object"]).columns
t_indep[categorical_columns] = t_indep[categorical_columns].astype("category")
test[categorical_columns] = test[categorical_columns].astype("category")

t_indep[categorical_columns] = t_indep[categorical_columns].apply(lambda x: x.cat.codes)
test[categorical_columns] = test[categorical_columns].apply(lambda x: x.cat.codes)

Em seguida, iremos normalizar os dados das variáveis independentes e de teste. Para fazer isso é feita a subtração da média de cada coluna e a divisão desse resultado pelo desvio padrão

In [29]:
t_indep = (t_indep - t_indep.mean()) / t_indep.std()
test = (test - test.mean()) / test.std()

## Criando o modelo

Para criar o modelo será utilizado o `RandomForestRegressor()` da biblioteca sklearn. Este modelo é baseado em árvores de decisão e é usado principalmente para problemas de regressão.

O funcionamento do RandomForestRegressor é semelhante ao de outros modelos de árvores de decisão. Ele cria uma floresta de árvores de decisão, cada uma com um conjunto diferente de features e exemplos de treinamento. As árvores são treinadas de forma independente e, no final, as previsões são feitas pela média das previsões de cada árvore.

Após iniciar o modelo, basta chamar o método `fit()`, que irá treiná-lo.

In [30]:
model = RandomForestRegressor()
model.fit(t_indep, t_dep)

Agora vamos ver como ficaram as previsões do modelo passando o arquivo de teste através do método `predict()`

In [32]:
predictions = model.predict(test)
predictions

array([125936.93, 153240.  , 184945.6 , ..., 156469.5 , 112103.5 ,
       245213.59])

Além disso, também é possível visualizar a precisão que o modelo obteve ao longo do treinamento através do método `score()` 

In [33]:
score = model.score(t_indep, t_dep)
score

0.9812032141970431

## Fazendo o Deploy

Agora, para que outras pessoas também possam testar o modelo, ele será submetido ao HuggingFace através de um arquivo model.pkl

In [38]:
import pickle

# Carregar o modelo treinado
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

O link para a aplicação está disponível aqui: https://huggingface.co/spaces/ErickMVdO/Lesson5